In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import *
import math
from datetime import datetime
import datawig
from tqdm import tqdm

In [ ]:
# Reading onshore wind speed data from csv file
wind_onshore_db = pd.read_csv(r'rawData\onshore_output\hourly_onshore_wind_speed.csv')
wind_onshore_db = wind_onshore_db.drop(["YYYYMMDD", "HH"], axis=1)
chosenStations = ["sta209","sta215","sta225","sta235","sta240","sta242","sta248","sta249","sta251","sta257","sta258","sta267","sta269","sta270","sta273","sta277","sta280","sta285","sta286","sta308","sta310","sta311","sta315","sta316","sta319","sta323","sta324","sta330","sta331","sta340","sta343","sta344","sta350","sta370","sta375"]

In [ ]:
stationList = []
for i in list(wind_onshore_db):
    if i in chosenStations:
        stationList.append(i)
        
wind_onshore_db = wind_onshore_db.loc[:,stationList] 

# Identifying the columns with missing values.

In [ ]:
missingRowSum = wind_onshore_db.isnull().sum(axis = 0)
blackList = []

# Identifying the columns with too many missing rows, and dropping them:
for i in range(len(list(wind_onshore_db))):
    # if higher than 50% of the total num of values:
    if missingRowSum[i] >= (len(wind_onshore_db) / 2):
        blackList.append(i)

wind_onshore_db = wind_onshore_db.drop(wind_onshore_db.columns[blackList], axis=1)


# Re-implementing the process above to identify columns with and without NaN
missingRowSum = wind_onshore_db.isnull().sum(axis = 0)
columnsWithNan = []
columnsWithoutNan = []

for i in range(len(list(wind_onshore_db))):
    # if column contains a NaN value:
    if missingRowSum[i] > 0:
        columnsWithNan.append(i)
    else:
        columnsWithoutNan.append(i)
        
# Identifying the column names for the columns with NaN and without NaN values
listOfColumnNamesWithNaN = wind_onshore_db.columns[columnsWithNan]
listOfColumnNamesWithoutNaN = wind_onshore_db.columns[columnsWithoutNan]

# Forming dataframe for imputer model: 

It takes about 30 minutes to process the imputation

In [ ]:
for i in listOfColumnNamesWithNaN:
    print(i)
    randomlyChosenFullColumns = np.random.choice(listOfColumnNamesWithoutNaN, 5, replace=False)
    df_for_imputation = pd.DataFrame(data= wind_onshore_db[randomlyChosenFullColumns], columns= randomlyChosenFullColumns)
    df_for_imputation = df_for_imputation.join(wind_onshore_db.loc[:,i])
    
    df_train = df_for_imputation.iloc[:, :]
    df_test = df_for_imputation.iloc[:, :]
    outputColumn = list(df_for_imputation)[-1]

    imputer = datawig.SimpleImputer(
        input_columns=list(df_for_imputation)[:-1], # column(s) containing information about the column we want to impute
        output_column= outputColumn, # the column we'd like to impute values for
        output_path = 'imputer_model' # stores model data and metrics
        )

    #Fit an imputer model on the train data
    imputer.fit(train_df=df_train, num_epochs=50)
    #Impute missing values and return original dataframe with predictions
    imputed = imputer.predict(df_test)
    
    wind_onshore_db[outputColumn] = wind_onshore_db[outputColumn].fillna(imputed[outputColumn + "_imputed"])

In [ ]:
# To export the imputed dataframe.
wind_onshore_db.to_json(r'processedData\processed_onshore_output\imputed_wind_onshore_db.json',orient='split')

# Checkpoint 1

In [ ]:
wind_onshore_db = pd.read_json(r'processedData\processed_onshore_output\imputed_wind_onshore_db.json',orient='split')

In [ ]:
weights = [["Flevoland", 0.322644302], ["Fryslân", 0.151948693], ["Noord-Holland", 0.123334978], ["Zeeland", 0.116921559], ["Groningen", 0.096694623], ["Zuid-Holland", 0.087814504], ["Noord-Brabant", 0.058707449]]

In [ ]:
average_province_windspeed = pd.DataFrame(columns=["Flevoland","Fryslân","Noord-Holland","Zeeland","Groningen","Zuid-Holland","Noord-Brabant"])
average_province_windspeed["Flevoland"] = wind_onshore_db.loc[:,["sta258","sta269","sta273"]].mean(axis=1)
average_province_windspeed["Fryslân"] = wind_onshore_db.loc[:,["sta242","sta251","sta267","sta270","sta277","sta285"]].mean(axis=1)
average_province_windspeed["Noord-Holland"] = wind_onshore_db.loc[:,["sta209","sta225","sta235","sta240","sta248","sta249"]].mean(axis=1)
average_province_windspeed["Zeeland"] = wind_onshore_db.loc[:,["sta308","sta310","sta311","sta315","sta316","sta319","sta323","sta324","sta331"]].mean(axis=1)
average_province_windspeed["Groningen"] = wind_onshore_db.loc[:,["sta280","sta286"]].mean(axis=1)
average_province_windspeed["Zuid-Holland"] = wind_onshore_db.loc[:,["sta330","sta343","sta344"]].mean(axis=1)
average_province_windspeed["Noord-Brabant"] = wind_onshore_db.loc[:,["sta340","sta350","sta370","sta375"]].mean(axis=1)

# Converting 10m/s values to m/s values
for i in list(average_province_windspeed):
    average_province_windspeed.loc[:, i] = average_province_windspeed.loc[:, i] / 10

# Identifying the weights of each province based on their number of wind turbine installations
weights = [["Flevoland", 0.322644302], ["Fryslân", 0.151948693], ["Noord-Holland", 0.123334978], ["Zeeland", 0.116921559], ["Groningen", 0.096694623], ["Zuid-Holland", 0.087814504], ["Noord-Brabant", 0.058707449]]    

for i in range(len(weights)):
    provinceName = weights[i][0]
    provinceWeight = weights[i][1]
    
    average_province_windspeed.loc[:, provinceName] = average_province_windspeed.loc[:, provinceName] * provinceWeight

avg_wind_speed_Hourly = pd.DataFrame(columns=["Value"])
avg_wind_speed_Hourly["Value"] = average_province_windspeed.sum(axis=1)
avg_wind_speed_Hourly = avg_wind_speed_Hourly.loc[:87599, :]
avg_wind_speed_Hourly["Value"] = avg_wind_speed_Hourly.loc[:, "Value"] * 1.3069127954971998034505709638027

# extending it to 30 years
avg_wind_speed_Hourly = pd.concat([avg_wind_speed_Hourly, avg_wind_speed_Hourly, avg_wind_speed_Hourly], ignore_index=True)

In [ ]:
onshore_wind_values = [0,1,2,3,4, 5,  6,  7,  8,  9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,27,28,29]
onshore_power_curve = [0,0,0,1,113,270,489,800,1229,1742,2309,2850,3287,3363,3400,3400,3400,3400,3400,3400,3400,3400,3400,3400,3400,3400,0, 0, 0, 0]

onshore_output = pd.DataFrame(columns=["Value", "dayIndex"])
emptyListValue = []

for i in tqdm(range(len(avg_wind_speed_Hourly))):
    windSpeed = avg_wind_speed_Hourly.loc[i,"Value"]
    windOutput = round(np.interp(windSpeed, onshore_wind_values, onshore_power_curve),3)
    
    emptyListValue.append(windOutput)
    
onshore_output["Value"] = emptyListValue
#onshore_output["dayIndex"] = three_hourly_onshore_wind_speed["dayIndex"]
# finding the output of 1kW installation: 
onshore_output["Value"] = onshore_output.loc[:,"Value"] / max(onshore_power_curve)

In [ ]:
for d in tqdm(range(3650*3)):
    for h in range(24):
        avg_wind_speed_Hourly.loc[(d*24)+h, "dayIndex"] = d

representativeDays = [149, 336, 1255, 1381, 1414, 1486, 2239, 3284, 3462]
listHours = []

for y in range(3):
    for d in representativeDays:
        for h in range(24):
            listHours.append(24*(d+y*3650) + h)

avg_wind_speed_Hourly = avg_wind_speed_Hourly[avg_wind_speed_Hourly.index.isin(listHours)]
avg_wind_speed_Hourly = avg_wind_speed_Hourly.reset_index(drop=True)

In [ ]:
three_hourly_onshore_wind_speed = pd.DataFrame(columns=["Value"])

emptyListValue = []
emptyListDay = []

for i in tqdm(range(int(len(avg_wind_speed_Hourly)/3))):
    day = avg_wind_speed_Hourly.loc[1+(3*i),"dayIndex"]
    value = avg_wind_speed_Hourly.loc[1+(3*i),"Value"]
    emptyListValue.append(value)
    emptyListDay.append(day)
    
three_hourly_onshore_wind_speed["Value"] = emptyListValue
three_hourly_onshore_wind_speed["dayIndex"] = emptyListDay

In [ ]:
onshore_wind_values = [0,1,2,3,4, 5,  6,  7,  8,  9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,27,28,29]
onshore_power_curve = [0,0,0,1,113,270,489,800,1229,1742,2309,2850,3287,3363,3400,3400,3400,3400,3400,3400,3400,3400,3400,3400,3400,3400,0, 0, 0, 0]

onshore_output = pd.DataFrame(columns=["Value", "dayIndex"])
emptyListValue = []

for i in tqdm(range(len(three_hourly_onshore_wind_speed))):
    windSpeed = three_hourly_onshore_wind_speed.loc[i,"Value"]
    windOutput = round(np.interp(windSpeed, onshore_wind_values, onshore_power_curve),3)*3
    
    emptyListValue.append(windOutput)
    
onshore_output["Value"] = emptyListValue
onshore_output["dayIndex"] = three_hourly_onshore_wind_speed["dayIndex"]
# finding the output of 1kW installation: 
onshore_output["Value"] = onshore_output.loc[:,"Value"] / max(onshore_power_curve)

In [ ]:
daily_onshore_windspeed = pd.DataFrame(columns=["Value", "dayIndex"])
emptyList = []

for i in tqdm(range(int(len(onshore_output)/8))):
    daily_onshore_windspeed.loc[i, "Value"] = list(onshore_output.loc[(0 + 8*i):(7 + 8*i), "Value"])
    daily_onshore_windspeed.loc[i, "dayIndex"] = onshore_output.loc[(0 + 8*i), "dayIndex"]

daily_onshore_windspeed.to_json(r"processedData\processed_onshore_output\daily_3hourly_onshoreOutput.json", orient='split')